<a href="https://colab.research.google.com/github/edivandocouto/Spatial-Data-Analysis/blob/main/ct_mapbiomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Instalando as bibliotecas
!pip install -q geemap==0.16.0
!pip install earthengine-api
!pip install geopandas

     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 8.8 MB 51.6 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 98 kB 7.2 MB/s 
     |████████████████████████████████| 1.3 MB 62.0 MB/s 
     |████████████████████████████████| 130 kB 45.3 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 3.0 MB 46.7 MB/s 
     |████████████████████████████████| 3.4 MB 40.8 MB/s 
     |████████████████████████████████| 198 kB 72.2 MB/s 
     |████████████████████████████████| 99 kB 6.8 MB/s 
     |████████████████████████████████| 1.2 MB 60.5 MB/s 
     |████████████████████████████████| 128 kB 68.5 MB/s 
     |████████████████████████████████| 1.6 MB 58.7 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 9.8 MB 38.2 MB/s 
     |████████████████████████████████| 346 kB 60.7 MB/s 
     |███████████████████

In [9]:
#Importando as bibliotecas
import ee
import geopandas as gpd
import json
import geemap

In [10]:
##Montando o acesso ao google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
##Autenticação do GEE e Inicialização
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xVc32-rXcG68Mj5y7HKomdN-ZEUySWaY0Yu5Ay1HL_0&tc=4LEnlFrRw6E70dX3EgvrEHNZQ5t7HONAcwWd_acMCeg&cc=hViasAsV7hGT6sBZVjO7MeVdkkMPZlQgKOYDTyPDoXQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJrjbbY0St4WbvM77kN-BAN9VuQ0qccYcpiNAO3i7OQ-92AdVZpFUmw

Successfully saved authorization token.


In [ ]:
#Autenticar no Google Cloud
#from google.colab import auth
#auth.authenticate_user()

In [12]:
# Coleção 3 Mapbiomas bacia amazonica  
MapBiomasAmazon = (ee.Image('projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2'))
print(MapBiomasAmazon.getInfo())

{'type': 'Image', 'bands': [{'id': 'classification_1985', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1986', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1987', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1988', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 

In [13]:
bandlist = [f'classification_{year}' for year in range(1985,2021)]
bandlist

['classification_1985',
 'classification_1986',
 'classification_1987',
 'classification_1988',
 'classification_1989',
 'classification_1990',
 'classification_1991',
 'classification_1992',
 'classification_1993',
 'classification_1994',
 'classification_1995',
 'classification_1996',
 'classification_1997',
 'classification_1998',
 'classification_1999',
 'classification_2000',
 'classification_2001',
 'classification_2002',
 'classification_2003',
 'classification_2004',
 'classification_2005',
 'classification_2006',
 'classification_2007',
 'classification_2008',
 'classification_2009',
 'classification_2010',
 'classification_2011',
 'classification_2012',
 'classification_2013',
 'classification_2014',
 'classification_2015',
 'classification_2016',
 'classification_2017',
 'classification_2018',
 'classification_2019',
 'classification_2020']

In [14]:
MapBiomasAmazon = (ee.Image('projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2'))
print('Number of images:',MapBiomasAmazon.bandNames().size().getInfo())
amazonCollection = ee.ImageCollection(MapBiomasAmazon.bandNames().map(lambda b: MapBiomasAmazon.select(bandlist)))
print('Collection size:',amazonCollection.size().getInfo())

Number of images: 36
Collection size: 36


In [15]:
asset = 'projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2'
amazonBiomas = ee.Image(asset)

toList   = [1, 2, 3,  4,  5, 6,  7,  8,   9,  8, 9,  11,  6,  8, 9, 10]
fromList = [1, 4, 5, 6, 10, 12, 14, 22, 24, 25, 26, 27, 29, 30, 33, 34]

names = amazonBiomas.bandNames()
#names = names.filter(ee.Filter.inList('item',bandlist))

def reclass(band):
  return amazonBiomas.remap(fromList,toList,0,band).rename([band])

amazonBiomas_reclass = names.map(reclass)

In [16]:
amazonBiomas_reclass.getInfo()

[{'type': 'Image',
  'bands': [{'id': 'classification_1985',
    'data_type': {'type': 'PixelType',
     'precision': 'int',
     'min': 0,
     'max': 11},
    'dimensions': [141216, 117285],
    'crs': 'EPSG:4326',
    'crs_transform': [0.00026949458523585647,
     0,
     -81.23725625725228,
     0,
     -0.00026949458523585647,
     10.372038101972407]}],
  'version': 1632375433141026,
  'id': 'projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2',
  'properties': {'system:footprint': {'type': 'LinearRing',
    'coordinates': [[-63.69537156255461, -21.235769568181407],
     [-60.127558032033875, -21.23576955307544],
     [-52.99193089007511, -21.23576953059324],
     [-48.23484617748847, -21.235769564160556],
     [-43.18015404170988, -21.235769294498468],
     [-43.180169190429005, 10.372173172469656],
     [-48.53216399204116, 10.372173325148935],
     [-57.74901562508815, 10.372173289588734],
     [-63.10073595010061, 10.372173304097

In [17]:
PamAmazon = ee.ImageCollection(amazonBiomas_reclass)

In [18]:
for el in PamAmazon.getInfo()['features']:
  print(el['bands'][0]['id'])

classification_1985
classification_1986
classification_1987
classification_1988
classification_1989
classification_1990
classification_1991
classification_1992
classification_1993
classification_1994
classification_1995
classification_1996
classification_1997
classification_1998
classification_1999
classification_2000
classification_2001
classification_2002
classification_2003
classification_2004
classification_2005
classification_2006
classification_2007
classification_2008
classification_2009
classification_2010
classification_2011
classification_2012
classification_2013
classification_2014
classification_2015
classification_2016
classification_2017
classification_2018
classification_2019
classification_2020


In [19]:
PamAmazon.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'classification_1985',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 11},
     'dimensions': [141216, 117285],
     'crs': 'EPSG:4326',
     'crs_transform': [0.00026949458523585647,
      0,
      -81.23725625725228,
      0,
      -0.00026949458523585647,
      10.372038101972407]}],
   'version': 1632375433141026,
   'id': 'projects/mapbiomas-raisg/public/collection3/mapbiomas_raisg_panamazonia_collection3_integration_v2',
   'properties': {'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-63.69537156255461, -21.235769568181407],
      [-60.127558032033875, -21.23576955307544],
      [-52.99193089007511, -21.23576953059324],
      [-48.23484617748847, -21.235769564160556],
      [-43.18015404170988, -21.235769294498468],
      [-43.180169190429005, 10.372173172469656],
      [-48.53216399204116, 10.372173325148935],
      [-57

In [25]:
# Região de Interesse a ROI
roi = ee.FeatureCollection('projects/ee-edivandocouto/assets/AmazonBasin_hybas_sa')

In [28]:
##Crie um mapa interativo 
Map = geemap.Map()
Map = geemap.Map(location=[-0.095659800460687,-51.31188826592861,], zoom_start= 10)
##Visualizando a imagem
vis_ = {"bands": ['classification_1985'], "min": 0, "max": 11}
vis_params = {"palette": ['red','yellow','green'], "min": 0, "max": 11}
ls8 = ee.Image(PamAmazon.first())
imagem = ee.Image(PamAmazon.select('classification_1985').first())
Map.addLayer(ls8, vis_, 'Primeira imagem')
Map.addLayer(imagem, vis_params, 'classification_1985')
# ##Adicionando a feature  = Converte para Image (desenha o limite)
limite = ee.Image().paint(roi, 0, 2)
Map.addLayer(limite,{'palette':'black'}, 'ROI')
Map

Map(center=[-0.095659800460687, -51.31188826592861], controls=(WidgetControl(options=['position', 'transparent…

# Bacia Pam Amazonica para o mapbiomas coleção 3 

# import import south america borders

In [ ]:
south_america = ee.FeatureCollection('projects/ee-edivandocouto/assets/south_america')


In [ ]:
south_america.get

{'type': 'FeatureCollection',
 'columns': {'ABBREV': 'String',
  'ADM0_A3': 'String',
  'ADM0_DIF': 'Float',
  'ADMIN': 'String',
  'FIPS_10_': 'Float',
  'FeatureCla': 'String',
  'GDP_MD_EST': 'Float',
  'GEOU_DIF': 'Float',
  'ISO_A2': 'String',
  'ISO_A3': 'String',
  'ISO_N3': 'Float',
  'LEVEL': 'Float',
  'LabelRank': 'Long',
  'MAP_COLOR': 'Float',
  'NAME': 'String',
  'NAME_FORMA': 'String',
  'NAME_SORT': 'String',
  'POP_EST': 'Float',
  'POSTAL': 'String',
  'SOVEREIGNT': 'String',
  'SOV_A3': 'String',
  'ScaleRank': 'Long',
  'TERR_': 'String',
  'TYPE': 'String',
  'fid': 'Integer',
  'system:index': 'String'},
 'version': 1665486090258673,
 'id': 'projects/ee-edivandocouto/assets/south_america',
 'properties': {'system:asset_size': 478256},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'GeometryCollection',
    'geometries': [{'type': 'LineString',
      'coordinates': [[-68.6419791635098, -54.79877578198554],
       [-68.6419791635098, -54.7987713228845]]}

In [ ]:
# Coleção 7 Mapbiomas Brasil
MapBiomasBrazil = (ee.Image('projects/mapbiomas-workspace/public/collection7/mapbiomas_collection70_integration_v2'))
print(MapBiomasAmazon.getInfo())

{'type': 'Image', 'bands': [{'id': 'classification_1985', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1986', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1987', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -81.23725625725228, 0, -0.00026949458523585647, 10.372038101972407]}, {'id': 'classification_1988', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [141216, 117285], 'crs': 

In [ ]:
# ##Abrindo nosso mapa
# # Defina um método para exibir blocos de imagens do Earth Engine em um folium map
Map = geemap.Map()
# ##Centralizando a imagem
Map = geemap.Map(location=[-3.173182, -59.973598], zoom_start= 5)
# #Para ver uma visualização de satélite do Google como um mapa base
Map.add_basemap('HYBRID')
##Obtendo a imagem

Map.addLayer(MapBiomasBrazil.clip(roi), {'bands': ['classification_1985'], 'min': 1, 'max': 36})
# ##Adicionando a feature  = Converte para Image (desenha o limite)
limite = ee.Image().paint(roi, 0, 2)
Map.addLayer(limite,{'palette':'red'})
# ##Visualizar o mapa
display(Map)


Map(center=[-3.173182, -59.973598], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…